<a href="https://colab.research.google.com/github/SAHIL9581/Agentic_model_main/blob/main/updated_agentic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pandas google-generativeai langchain-google-genai langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import base64
import getpass
import os
from IPython.display import display, Image
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")


Enter your Google AI API key: ··········


In [8]:
# Step 4: Initialize the model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


In [13]:
from google.colab import files  # Make sure this is included if using Colab

def analyze_data():
    print("\nPlease upload your CSV file:")
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded. Returning to main menu.")
        return

    filename = next(iter(uploaded))

    try:
        # Try different encodings if needed
        try:
            df = pd.read_csv(filename, on_bad_lines='skip')
        except UnicodeDecodeError:
            df = pd.read_csv(filename, on_bad_lines='skip', encoding='latin1')

        print(f"\n✅ Successfully loaded '{filename}' (Shape: {df.shape})")
        print("\nFirst 5 rows:")
        display(df.head())

        max_rows_to_send = min(100, len(df))  # Don't exceed dataframe length
        data_sample = df.head(max_rows_to_send).to_string(index=False)

        print("\nYou can ask questions about your data. Type 'back' to return to the main menu.")
        print("Examples:")
        print("- What are the column names and what do they represent?")
        print("- Are there any missing values in the data?")
        print("- What insights can you derive from this data?")

        while True:
            question = input("\n💬 Enter your question about the dataset (or 'back' to return): ")

            if question.lower() == 'back':
                break

            prompt = f"""
            You are a helpful data analyst. Be concise but thorough.

            Dataset preview (first {max_rows_to_send} rows):

            {data_sample}

            Question: {question}

            Provide your analysis based on this data. If the question can't be answered
            from the data, explain why and suggest what additional data would be needed.
            """

            print("\n🔍 Analyzing your data...")
            response = llm.invoke(prompt)

            print("\n🤖 Analysis Results:\n")
            print(response.content)

    except Exception as e:
        print(f"❌ Error processing file: {e}")


In [14]:
def main():
    while True:
        print("\n===== Gemini AI Assistant =====")
        print("1. Analyze CSV Data")
        print("2. Chat with Gemini")
        print("3. Exit")

        choice = input("Select an option (1-3): ").strip()

        if choice == "1":
            analyze_data()
        elif choice == "2":
            question = input("\n💬 Enter your question: ").strip()
            print("\n🤖 Thinking...")
            response = llm.invoke(question)
            print("\nResponse:\n")
            print(response.content)
        elif choice == "3":
            print("\nGoodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


===== Gemini AI Assistant =====
1. Analyze CSV Data
2. Chat with Gemini
3. Exit
Select an option (1-3): 1

Please upload your CSV file:


Saving Well data.CSV to Well data (2).CSV

✅ Successfully loaded 'Well data (2).CSV' (Shape: (15406, 47))

First 5 rows:


,WSN,UWI (APINum),Well Number,Well Name,Well Label,Sym Code,Operator,Hist Oper,Lease Name,Lease Nbr,...,WELL\r CUMWTR,WELL\r WHIPSTOCK,WELL\r COMP_DATE,WELL\r SPUD_DATE,WELL\r PERMIT_DATE,WELL\r ABAND_DATE,WELL\r REPORT_DATE,WELL\r RPT_DATE,WELL\r LAST_ACT_DATE,WELL\r TD_DATE
0,2492,42001004370000,6-B,NaN,4.200100e+13,PLUGGAS,YOUNG MARSHALL R DRL,ROESER & PENDLETON,L C BILLETT,6-B,...,NaN,NaN,11/16/1943,08/09/1943,07/30/1943,11/08/1961,04/01/1977,NaN,06/30/2022,NaN
1,2493,42001004380000,5-B,NaN,4.200100e+13,TA-OIL,ROESER & PENDLETON,ROESER & PENDLETON,BILLETT L R,5-B,...,NaN,NaN,08/25/1936,08/04/1936,07/25/1936,NaN,12/29/1992,NaN,06/30/2022,NaN
2,2706,42001300900000,SWD-1,NaN,4.200130e+13,SWDOP,ENR OPERATING LLC,FAULCONER VERNON E,WATHEN BEN H,SWD-1,...,NaN,NaN,10/17/1969,10/10/1969,09/22/1969,NaN,11/11/2009,NaN,11/02/2023,NaN
3,2712,42001300950000,1,NaN,4.200130e+13,DRY,SPENCE RALPH,SPENCE RALPH,EZEM G SCARBOROUGH,1,...,NaN,NaN,11/27/1969,10/24/1969,10/14/1969,NaN,12/01/1969,NaN,04/14/2021,NaN
4,2741,42001301360000,1,NaN,4.200130e+13,DRY,FARISH W S,FARISH W S,O L ELLIS,1,...,NaN,NaN,02/15/1970,02/05/1970,01/26/1970,NaN,03/01/1970,NaN,02/24/2015,NaN



You can ask questions about your data. Type 'back' to return to the main menu.
Examples:
- What are the column names and what do they represent?
- Are there any missing values in the data?
- What insights can you derive from this data?

💬 Enter your question about the dataset (or 'back' to return):  What are the column names and what do they represent?

🔍 Analyzing your data...

🤖 Analysis Results:

The provided data preview shows a portion of a well dataset.  Here's a description of the column names based on their likely meaning:

* **WSN:** Well Serial Number (unique identifier).
* **UWI (APINum):** Unique Well Identifier (API Number), another unique identifier.
* **Well Number:** Well number assigned by the operator or regulatory agency.
* **Well Name:**  Name given to the well by the operator.  Often descriptive (e.g., location).
* **Well Label:**  A label or designation for the well.  May be redundant with Well Name.
* **Sym Code:** Symbolic code representing well type or status.